# Reconstructing the Full Test Images

Now that we trained the model and predicted what the test crops will look like, it is time to focus on reconstruction. Using our predicted crops, we will stitch them back together to create complete masks we can observe.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

The following function is all we need this time:

In [3]:
# This does all the heavylifting and recreates the images
# Instead of averaging, each pixel is determined by what percentage of the crops outline that pixel as white, by the variable vote_percentage

def reconstruct_images_from_crops(crops_dir, save_dir, original_size=(512, 512), crop_size=(48, 48), stride=16, crops_per_image=900, vote_percentage=0.5):
    os.makedirs(save_dir, exist_ok=True)

    # Get the list of crop files and sort them
    crop_files = sorted([f for f in os.listdir(crops_dir) if f.endswith('.jpg')])

    # Extract unique base names by cutting off the '_crop' at the end
    base_names = {os.path.splitext(f)[0].rsplit('_crop', 1)[0] for f in crop_files}

    # Iterate through each base name
    for base_name in base_names:
        print(f"Processing base name: {base_name}")  # Debug statement

        # Create arrays for accumulating the image and counting overlaps
        vote_array = np.zeros((original_size[0], original_size[1]), dtype=np.int32)  # ADDED
        overlap_count = np.zeros((original_size[0], original_size[1]), dtype=np.int32)

        # Iterate through the crop files for the current base name
        for crop_number in range(1, crops_per_image + 1):
            crop_file = f"{base_name}_crop{crop_number}.jpg"
            if crop_file not in crop_files:
                continue  # Skip if the crop file does not exist

            crop_path = os.path.join(crops_dir, crop_file)
            crop = np.array(Image.open(crop_path).convert('L'))  # ADJUSTED: Convert to grayscale

            # Calculate the position in the original image
            num_crops_per_row = (original_size[1] - crop_size[1]) // stride + 1
            row = (crop_number - 1) // num_crops_per_row
            col = (crop_number - 1) % num_crops_per_row
            y = row * stride
            x = col * stride

            # Accumulate the vote in the vote_array
            vote_array[y:y+crop_size[0], x:x+crop_size[1]] += (crop > 127).astype(np.int32)  # ADDED: Count votes
            overlap_count[y:y+crop_size[0], x:x+crop_size[1]] += 1

        # Apply majority voting with 50% threshold
        vote_threshold = (overlap_count * vote_percentage).astype(np.int32)  # ADDED: Calculate threshold based on percentage
        final_image = (vote_array > vote_threshold).astype(np.uint8) * 255  # ADDED: Apply voting threshold and convert to binary image

        # Save the reconstructed image
        save_path = os.path.join(save_dir, f"{base_name}_reconstructed.jpg")
        Image.fromarray(final_image).save(save_path, format='JPEG')
        print(base_name, "reconstructed")

In [4]:
# Reconstruct Test Images
base_path = '/content/drive/My Drive/ML_projects/VesselProject/RetinalImages'
predicted_masks_dir = os.path.join(base_path, 'Predicted_masks')
save_reconstructed_dir = os.path.join(base_path, 'Reconstructed_masks')
reconstruct_images_from_crops(predicted_masks_dir, save_reconstructed_dir, crop_size=(128, 128), stride=32, crops_per_image=169, vote_percentage=0.5)

Processing base name: 33_DRIVE
33_DRIVE reconstructed
Processing base name: Image_01R_CHASE
Image_01R_CHASE reconstructed
Processing base name: 13_g_HRF
13_g_HRF reconstructed
Processing base name: Image_02L_CHASE
Image_02L_CHASE reconstructed
Processing base name: Image_03L_CHASE
Image_03L_CHASE reconstructed
Processing base name: 15_dr_HRF
15_dr_HRF reconstructed
Processing base name: 37_DRIVE
37_DRIVE reconstructed
Processing base name: 14_g_HRF
14_g_HRF reconstructed
Processing base name: Image_03R_CHASE
Image_03R_CHASE reconstructed
Processing base name: 15_h_HRF
15_h_HRF reconstructed
Processing base name: 13_h_HRF
13_h_HRF reconstructed
Processing base name: 15_g_HRF
15_g_HRF reconstructed
Processing base name: 14_h_HRF
14_h_HRF reconstructed
Processing base name: 36_DRIVE
36_DRIVE reconstructed
Processing base name: 13_dr_HRF
13_dr_HRF reconstructed
Processing base name: 14_dr_HRF
14_dr_HRF reconstructed
Processing base name: 35_DRIVE
35_DRIVE reconstructed
Processing base name

## Show the Reconstruction

In addition, this will show what all 20 test masks look like next to their ground truth counterparts

In [5]:
base_path = '/content/drive/My Drive/ML_projects/VesselProject/RetinalImages'
image_dir = os.path.join(base_path, 'Test', 'Images')
mask_dir = os.path.join(base_path, 'Processed', 'Test', 'Masks')
reconstructed_dir = os.path.join(base_path, 'Reconstructed_masks')

# Get the list of reconstructed images
reconstructed_images = [f for f in os.listdir(reconstructed_dir) if f.endswith('_reconstructed.jpg')]

acc_list = []

# Iterate through the reconstructed images
for reconstructed_image in reconstructed_images:
    # Extract the base name
    base_name = reconstructed_image.split('_reconstructed')[0]

    # Load the corresponding image, mask, and reconstructed image
    image_path = next((os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.startswith(base_name)), None)
    mask_path = next((os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.startswith(base_name)), None)
    reconstructed_path = os.path.join(reconstructed_dir, reconstructed_image)

    image = Image.open(image_path)
    mask = Image.open(mask_path).convert('L')  # Convert mask to grayscale
    reconstructed = Image.open(reconstructed_path)

    mask_np = np.array(mask)
    reconstructed_np = np.array(reconstructed)

    # Calculate accuracy
    correct_predictions = np.sum(mask_np == reconstructed_np)
    total_pixels = mask_np.size
    accuracy = correct_predictions / total_pixels
    print(f"Accuracy for {base_name}: {accuracy:.4f}")
    acc_list.append(accuracy)

    # Display the images side by side
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(image)
    axes[0].set_title('Original Image')
    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title('Ground Truth Mask')
    axes[2].imshow(reconstructed, cmap='gray')
    axes[2].set_title('Reconstructed Mask')

    plt.tight_layout()
    plt.show()

print('AVERAGE ACCURACY:', np.mean(acc_list))


Output hidden; open in https://colab.research.google.com to view.

## There you have it! Thank you for reviewing my project!

As you can see, the reconstructed masks capture most of the large vessels well and mantain an accuracy close to 80%. This makes it very useful and impactful in the analysis of retinal images

Unfortunately, it does have a weekness in identifying the smaller vessels, and in a few cases it introduces some distracting noise. However, the visuals are mostly useful and heavily reflect the vessels of the orignal images.

Thank you very much, and stay tuned to my github for more excited projects in the future!